/path/open_video_source.py
import cv2

def open_video_source(source=0):
    """
    source: int for camera, str for video file path.
    """
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video source: {source}")
    return cap

# Usage
cap = open_video_source(0)  # Camera
# OR
cap = open_video_source('/path/to/file.mp4')  # Video file

In [1]:
# My Abstract Video Class and Concrete Cameras

import cv2
import numpy as np
from abc import ABC, abstractmethod

class AbstractCamera(ABC):
    def __init__(self, video_path, name=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        self.name = name or self.__class__.__name__
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    @abstractmethod
    def read_frame(self, frame_num):
        pass

    def release(self):
        self.cap.release()

    def __repr__(self):
        return f"<{self.name} path={self.video_path} frames={self.frame_count} fps={self.fps}>"

class Insta360Camera(AbstractCamera):
    def read_frame(self, frame_num):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.cap.read()
        # Custom 360 correction (if needed) goes here
        return frame if ret else None

class NikonCamera(AbstractCamera):
    def read_frame(self, frame_num):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.cap.read()
        # Any Nikon-specific corrections go here
        return frame if ret else None

# Example Usage
if __name__ == "__main__":
    insta360 = Insta360Camera('/path/to/insta360.mp4', name='Insta360')
    nikon = NikonCamera('/path/to/nikon.mp4', name='Nikon')

    # Get a frame from both at (say) frame 100
    frame_insta = insta360.read_frame(100)
    frame_nikon = nikon.read_frame(100)

    print(insta360)
    print(nikon)

    # You can now compare, analyze, or pass these to Blender workflows

    insta360.release()
    nikon.release()

AttributeError: module 'cv2' has no attribute 'VideoCapture'

In [2]:
help(cv2)

Help on built-in module cv2:

NAME
    cv2 - Python wrapper for OpenCV.

SUBMODULES
    Error
    aruco
    barcode
    cuda
    detail
    dnn
    fisheye
    flann
    ipp
    ml
    ocl
    ogl
    parallel
    samples
    segmentation
    utils

CLASSES
    builtins.Exception(builtins.BaseException)
        error
    builtins.object
        Algorithm
            AlignExposures
                AlignMTB
            BackgroundSubtractor
                BackgroundSubtractorKNN
                BackgroundSubtractorMOG2
            BaseCascadeClassifier
            CLAHE
            CalibrateCRF
                CalibrateDebevec
                CalibrateRobertson
            DenseOpticalFlow
                DISOpticalFlow
                FarnebackOpticalFlow
                VariationalRefinement
            DescriptorMatcher
                BFMatcher
                FlannBasedMatcher
            GeneralizedHough
                GeneralizedHoughBallard
                GeneralizedHoughGuil
 

In [ ]:
# Used to Sync Footage across same time by means of histograph matching

import cv2
import numpy as np

# --- CONFIGURATION ---
video1_path = '/path/to/insta360_clip.mp4'
video2_path = '/path/to/nikon_clip.mp4'
max_offset = 300  # Max frames to search ahead/behind (tune to your case)
stride = 3        # Only analyze every nth frame for speed

def frame_histogram(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [64], [0,256])
    return cv2.normalize(hist, hist).flatten()

def video_histograms(path, stride=1, max_frames=300):
    cap = cv2.VideoCapture(path)
    hists = []
    count = 0
    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if count % stride == 0:
            hists.append(frame_histogram(frame))
        count += 1
    cap.release()
    return np.array(hists)

# Step 1: Compute histograms for both videos
hist1 = video_histograms(video1_path, stride=stride, max_frames=max_offset)
hist2 = video_histograms(video2_path, stride=stride, max_frames=max_offset)

# Step 2: Search for the offset with highest histogram similarity
min_len = min(len(hist1), len(hist2))
best_offset = 0
best_score = -np.inf

for offset in range(-max_offset, max_offset, stride):
    if offset < 0:
        h1 = hist1[:min_len + offset]
        h2 = hist2[-offset:min_len]
    else:
        h1 = hist1[offset:min_len]
        h2 = hist2[:min_len - offset]
    if len(h1) < 10 or len(h2) < 10:
        continue
    scores = [cv2.compareHist(h1[i], h2[i], cv2.HISTCMP_CORREL) for i in range(len(h1))]
    avg_score = np.mean(scores)
    if avg_score > best_score:
        best_score = avg_score
        best_offset = offset

print(f"Best frame offset (video2 relative to video1): {best_offset * stride} frames")

# Making Abstract Video & Camera Class System With Integrated Workflow Example 

In [ ]:
import os
import sys
from abc import ABC, abstractmethod

# ----------------------
# 1. USER-SOURCED FILE/DIR PICKER (MULTI-PLATFORM)
# ----------------------

def select_video_dirs(max_dirs=3):
    """
    Let user select multiple directories interactively.
    Returns a list of directory paths.
    """
    dirs = []
    try:
        import tkinter as tk
        from tkinter import filedialog
        tk.Tk().withdraw()
        for _ in range(max_dirs):
            d = filedialog.askdirectory(mustexist=True, title="Select video folder")
            if not d: break
            dirs.append(d)
    except Exception:
        print("Tkinter not available or running headless. Skipping GUI directory picker.")
    return dirs

def list_videos_in_dirs(dirs, extensions=('.mp4', '.mov', '.avi', '.mkv')):
    """
    List all video files in given directories.
    Returns a list of file paths.
    """
    videos = []
    for d in dirs:
        try:
            for f in os.listdir(d):
                if f.lower().endswith(extensions):
                    videos.append(os.path.join(d, f))
        except Exception as e:
            print(f"Could not list directory {d}: {e}")
    return videos

def get_video_file_interactive():
    """
    Cross-platform: notebook upload, iOS/Juno auto-find, or desktop file picker.
    Returns a valid video file path, or raises if none found.
    """
    # 1. Notebook (Jupyter upload widget)
    try:
        from IPython.display import display
        import ipywidgets as widgets
        upload_widget = widgets.FileUpload(accept='.mp4,.mov,.avi,.mkv', multiple=False)
        display(upload_widget)
        print("Upload your video file in the widget above, then rerun this cell.")
        return None
    except Exception:
        pass

    # 2. iOS/Juno/CLI: latest video file in cwd
    cwd = os.getcwd()
    files = [f for f in os.listdir(cwd) if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
    if files:
        files.sort(key=lambda x: os.path.getmtime(os.path.join(cwd, x)), reverse=True)
        print(f"Auto-selected latest video: {files[0]}")
        return os.path.join(cwd, files[0])

    # 3. Desktop (Tkinter file picker)
    try:
        import tkinter as tk
        from tkinter import filedialog
        tk.Tk().withdraw()
        video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.mov *.avi *.mkv")])
        if video_path:
            return video_path
    except Exception:
        pass

    raise Exception("No video file selected or found in any method!")

# ----------------------
# 2. ABSTRACT VIDEO CAMERA CLASS SYSTEM
# ----------------------

import cv2

class AbstractCamera(ABC):
    """
    Abstract base class for video cameras (local file, network file, etc).
    """

    def __init__(self, video_path, name=None):
        self.video_path = video_path
        self.name = name or self.__class__.__name__
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise IOError(f"Cannot open video: {video_path}")
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    @abstractmethod
    def read_frame(self, frame_num):
        pass

    def release(self):
        self.cap.release()

    def __repr__(self):
        return (f"<{self.name} path={self.video_path} frames={self.frame_count} "
                f"fps={self.fps} size={self.width}x{self.height}>")

class Insta360Camera(AbstractCamera):
    def read_frame(self, frame_num):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.cap.read()
        # Add custom 360 correction here if needed
        return frame if ret else None

class NikonCamera(AbstractCamera):
    def read_frame(self, frame_num):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.cap.read()
        # Add Nikon-specific corrections here if needed
        return frame if ret else None

# ----------------------
# 3. INTEGRATED WORKFLOW EXAMPLE
# ----------------------

if __name__ == "__main__":
    # Step 1: Let user select directories to maintain "context" and minimize data copying
    video_dirs = select_video_dirs(max_dirs=3)
    if not video_dirs:
        print("No directories selected. Falling back to single file interactive selection...")
        video_path = get_video_file_interactive()
        camera = Insta360Camera(video_path)
        print(camera)
        frame = camera.read_frame(100)
        # Do something with frame...
        camera.release()
    else:
        # Step 2: List all video files (metadata only)
        video_files = list_videos_in_dirs(video_dirs)
        print(f"Found {len(video_files)} video files in selected dirs.")

        # Step 3: Lazy-load files as needed
        for i, path in enumerate(video_files):
            print(f"[{i}] {os.path.basename(path)}")
        if video_files:
            # For demo, pick the first file and open it with a concrete camera
            selected_path = video_files[0]
            camera = NikonCamera(selected_path)
            print(camera)
            frame = camera.read_frame(100)
            # Do something with frame...
            camera.release()

# --- END OF FILE ---

# Integrated Piggyback Camera Automation App
### Combines multi-camera video processing with Blender automation

In [3]:
#!/usr/bin/env python3
"""
Integrated Piggyback Camera Automation App
Combines multi-camera video processing with Blender automation
"""

#import tkinter as tk
#from tkinter import ttk, filedialog, messagebox
import subprocess
import json
import threading
import queue
import os
import sys
import cv2
import tempfile
import numpy as np
from pathlib import Path
import time
from datetime import datetime
from abc import ABC, abstractmethod

# Graceful Fail GUI

def try_import_tkinter():
    """Returns (tk, ttk, filedialog, messagebox) or None if unavailable."""
    try:
        import tkinter as tk
        from tkinter import ttk, filedialog, messagebox
        return tk, ttk, filedialog, messagebox
    except ImportError:
        return None, None, None, None

def get_app_mode():
    """Detects the best UI mode: 'tkinter', 'jupyter', or 'cli'."""
    # Try Tkinter
    tk, ttk, filedialog, messagebox = try_import_tkinter()
    if tk is not None:
        return 'tkinter'
    # Jupyter?
    try:
        get_ipython
        return 'jupyter'
    except NameError:
        pass
    # Default: CLI
    return 'cli'


# ----------------------
# INTEGRATED VIDEO CAMERA SYSTEM
# ----------------------

def select_video_dirs(max_dirs=3, parent=None):
    """Let user select multiple directories interactively."""
    dirs = []
    try:
        for _ in range(max_dirs):
            d = filedialog.askdirectory(
                parent=parent,
                mustexist=True, 
                title=f"Select video folder ({len(dirs)+1}/{max_dirs})"
            )
            if not d: 
                break
            dirs.append(d)
    except Exception as e:
        print(f"Directory selection error: {e}")
    return dirs

def list_videos_in_dirs(dirs, extensions=('.mp4', '.mov', '.avi', '.mkv', '.invis')):
    """List all video files in given directories."""
    videos = []
    for d in dirs:
        try:
            for f in os.listdir(d):
                if f.lower().endswith(extensions):
                    videos.append(os.path.join(d, f))
        except Exception as e:
            print(f"Could not list directory {d}: {e}")
    return videos

class AbstractCamera(ABC):
    """Abstract base class for video cameras."""
    
    def __init__(self, video_path, name=None):
        self.video_path = video_path
        self.name = name or self.__class__.__name__
        self.metadata = self._extract_metadata()
        
    def _extract_metadata(self):
        """Extract video metadata using OpenCV."""
        try:
            cap = cv2.VideoCapture(self.video_path)
            if not cap.isOpened():
                return None
                
            metadata = {
                'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
                'fps': cap.get(cv2.CAP_PROP_FPS),
                'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                'duration': 0
            }
            if metadata['fps'] > 0:
                metadata['duration'] = metadata['frame_count'] / metadata['fps']
            
            cap.release()
            return metadata
        except Exception as e:
            print(f"Error extracting metadata from {self.video_path}: {e}")
            return None
    
    @abstractmethod
    def extract_tracking_data(self):
        """Extract camera tracking data."""
        pass
    
    @abstractmethod
    def read_frame(self, frame_num):
        """Read specific frame from video."""
        pass
    
    def get_info_string(self):
        """Get formatted info string for display."""
        if not self.metadata:
            return f"{self.name}: Metadata unavailable"
        
        m = self.metadata
        return (f"{self.name}: {m['width']}x{m['height']} @ {m['fps']:.1f}fps, "
                f"{m['frame_count']} frames ({m['duration']:.1f}s)")

class Insta360Camera(AbstractCamera):
    """Insta360 camera with tracking data extraction."""
    
    def extract_tracking_data(self):
        """Extract tracking data from Insta360 file."""
        # For .invis files, implement actual extraction
        if self.video_path.lower().endswith('.invis'):
            return self._extract_invis_tracking()
        else:
            # For regular video files, use motion analysis
            return self._analyze_motion_tracking()
    
    def _extract_invis_tracking(self):
        """Extract tracking from .invis file."""
        # Placeholder for actual Insta360 SDK integration
        # In practice, you'd use Insta360 Studio CLI or SDK
        print(f"Extracting tracking from .invis file: {self.video_path}")
        
        # Simulate tracking data
        frames = []
        if self.metadata:
            for i in range(min(self.metadata['frame_count'], 1000)):  # Limit for demo
                frames.append({
                    'frame': i,
                    'timestamp': i / self.metadata['fps'] if self.metadata['fps'] > 0 else i / 30,
                    'position': {
                        'x': np.sin(i * 0.01) * 2,
                        'y': i * 0.005,
                        'z': np.cos(i * 0.01) * 0.5
                    },
                    'rotation': {
                        'x': np.sin(i * 0.02) * 10,
                        'y': i * 0.1,
                        'z': np.cos(i * 0.015) * 5
                    },
                    'confidence': 0.95
                })
        
        return {'frames': frames, 'source': 'insta360_invis'}
    
    def _analyze_motion_tracking(self):
        """Analyze motion from regular video file."""
        print(f"Analyzing motion tracking from video: {self.video_path}")
        
        # Use OpenCV for basic motion analysis
        cap = cv2.VideoCapture(self.video_path)
        frames = []
        
        try:
            frame_num = 0
            prev_gray = None
            
            while frame_num < min(self.metadata['frame_count'], 300):  # Limit processing
                ret, frame = cap.read()
                if not ret:
                    break
                
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                if prev_gray is not None:
                    # Calculate optical flow for motion estimation
                    flow = cv2.calcOpticalFlowPyrLK(
                        prev_gray, gray, 
                        np.array([[frame.shape[1]//2, frame.shape[0]//2]], dtype=np.float32),
                        None
                    )[0]
                    
                    # Convert flow to simple position estimate
                    motion_x = float(flow[0][0] - frame.shape[1]//2) * 0.001
                    motion_y = float(flow[0][1] - frame.shape[0]//2) * 0.001
                else:
                    motion_x = motion_y = 0
                
                frames.append({
                    'frame': frame_num,
                    'timestamp': frame_num / self.metadata['fps'] if self.metadata['fps'] > 0 else frame_num / 30,
                    'position': {'x': motion_x, 'y': motion_y, 'z': 0},
                    'rotation': {'x': 0, 'y': 0, 'z': 0},
                    'confidence': 0.7
                })
                
                prev_gray = gray
                frame_num += 1
        
        finally:
            cap.release()
        
        return {'frames': frames, 'source': 'opencv_motion'}
    
    def read_frame(self, frame_num):
        """Read specific frame."""
        cap = cv2.VideoCapture(self.video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        cap.release()
        return frame if ret else None

class MainCamera(AbstractCamera):
    """Main camera (hero cam) for primary footage."""
    
    def extract_tracking_data(self):
        """Main cameras typically don't provide tracking data."""
        return None
    
    def read_frame(self, frame_num):
        """Read specific frame."""
        cap = cv2.VideoCapture(self.video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        cap.release()
        return frame if ret else None

class GoProCamera(MainCamera):
    """GoPro camera - a type of main camera."""
    pass

class NikonCamera(AbstractCamera):
    """Nikon camera for high-quality footage."""
    
    def extract_tracking_data(self):
        """Nikon cameras don't typically provide tracking data."""
        return None
    
    def read_frame(self, frame_num):
        """Read specific frame with Nikon-specific processing."""
        cap = cv2.VideoCapture(self.video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        cap.release()
        return frame if ret else None

# ----------------------
# CAMERA FACTORY
# ----------------------

def create_camera(video_path):
    """Factory function to create appropriate camera type."""
    filename = os.path.basename(video_path).lower()
    
    if '.invis' in filename or 'insta360' in filename:
        return Insta360Camera(video_path, f"Insta360({os.path.basename(video_path)})")
    elif 'gopro' in filename or 'hero' in filename:
        return GoProCamera(video_path, f"GoPro({os.path.basename(video_path)})")
    elif any(x in filename for x in ['nikon', 'dslr', '.nef']):
        return NikonCamera(video_path, f"Nikon({os.path.basename(video_path)})")
    else:
        # Default to MainCamera for unknown types
        return MainCamera(video_path, f"MainCam({os.path.basename(video_path)})")

# ----------------------
# MAIN APPLICATION
# ----------------------

class IntegratedPiggybackApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Integrated Piggyback Camera Processor")
        self.root.geometry("900x1000")
        
        # Processing queue for background tasks
        self.process_queue = queue.Queue()
        self.is_processing = False
        
        # Video management
        self.video_directories = []
        self.available_videos = []
        self.selected_cameras = {'tracking': None, 'main': None}
        
        # Configuration
        self.config = {
            'physical_offset': {'x': 0.0, 'y': 0.0, 'z': -0.12},
            'blender_path': 'blender',
            'output_dir': str(Path.home() / 'PiggybackOutput'),
            'render_settings': {
                'resolution_x': 1920,
                'resolution_y': 1080,
                'fps': 30,
                'format': 'PNG'
            }
        }
        
        self.setup_ui()
        
    def setup_ui(self):
        """Create the user interface."""
        
        # Main notebook for tabs
        notebook = ttk.Notebook(self.root)
        notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Video Selection Tab
        video_frame = ttk.Frame(notebook)
        notebook.add(video_frame, text="Video Selection")
        self.setup_video_tab(video_frame)
        
        # Camera Setup Tab
        camera_frame = ttk.Frame(notebook)
        notebook.add(camera_frame, text="Camera Setup")
        self.setup_camera_tab(camera_frame)
        
        # Processing Tab
        process_frame = ttk.Frame(notebook)
        notebook.add(process_frame, text="Processing")
        self.setup_processing_tab(process_frame)
        
        # Results Tab
        results_frame = ttk.Frame(notebook)
        notebook.add(results_frame, text="Results")
        self.setup_results_tab(results_frame)
    
    def setup_video_tab(self, parent):
        """Setup video selection and management."""
        
        # Directory selection
        ttk.Label(parent, text="Video Source Directories:").pack(pady=(10,5))
        
        dir_frame = ttk.Frame(parent)
        dir_frame.pack(fill='x', padx=10, pady=5)
        
        ttk.Button(dir_frame, text="Add Video Directories", 
                  command=self.add_video_directories).pack(side='left', padx=5)
        ttk.Button(dir_frame, text="Scan for Videos", 
                  command=self.scan_videos).pack(side='left', padx=5)
        ttk.Button(dir_frame, text="Clear All", 
                  command=self.clear_videos).pack(side='left', padx=5)
        
        # Directory list
        self.dir_listbox = tk.Listbox(parent, height=4)
        self.dir_listbox.pack(fill='x', padx=10, pady=5)
        
        # Available videos
        ttk.Label(parent, text="Available Videos:").pack(pady=(20,5))
        
        # Video list with details
        video_list_frame = ttk.Frame(parent)
        video_list_frame.pack(fill='both', expand=True, padx=10, pady=5)
        
        # Treeview for video details
        columns = ('type', 'duration', 'resolution', 'fps')
        self.video_tree = ttk.Treeview(video_list_frame, columns=columns, show='tree headings')
        
        self.video_tree.heading('#0', text='Video File')
        self.video_tree.heading('type', text='Type')
        self.video_tree.heading('duration', text='Duration')
        self.video_tree.heading('resolution', text='Resolution')
        self.video_tree.heading('fps', text='FPS')
        
        self.video_tree.column('#0', width=300)
        self.video_tree.column('type', width=100)
        self.video_tree.column('duration', width=80)
        self.video_tree.column('resolution', width=100)
        self.video_tree.column('fps', width=60)
        
        video_scrollbar = ttk.Scrollbar(video_list_frame, orient='vertical', command=self.video_tree.yview)
        self.video_tree.configure(yscrollcommand=video_scrollbar.set)
        
        self.video_tree.pack(side='left', fill='both', expand=True)
        video_scrollbar.pack(side='right', fill='y')
        
        # Video selection buttons
        selection_frame = ttk.Frame(parent)
        selection_frame.pack(fill='x', padx=10, pady=10)
        
        ttk.Button(selection_frame, text="Set as Tracking Camera", 
                  command=self.set_tracking_camera).pack(side='left', padx=5)
        ttk.Button(selection_frame, text="Set as Main Camera", 
                  command=self.set_main_camera).pack(side='left', padx=5)
        ttk.Button(selection_frame, text="Analyze Selected", 
                  command=self.analyze_selected_video).pack(side='left', padx=5)
    
    def setup_camera_tab(self, parent):
        """Setup camera configuration."""
        
        # Selected cameras display
        ttk.Label(parent, text="Piggyback Rig Configuration:").pack(pady=10)
        
        # Tracking camera info
        tracking_frame = ttk.LabelFrame(parent, text="Tracking Camera (Top)")
        tracking_frame.pack(fill='x', padx=10, pady=5)
        
        self.tracking_info_var = tk.StringVar(value="No tracking camera selected")
        ttk.Label(tracking_frame, textvariable=self.tracking_info_var, wraplength=600).pack(pady=10)
        
        # Main camera info
        main_frame = ttk.LabelFrame(parent, text="Main Camera (Bottom)")
        main_frame.pack(fill='x', padx=10, pady=5)
        
        self.main_info_var = tk.StringVar(value="No main camera selected")
        ttk.Label(main_frame, textvariable=self.main_info_var, wraplength=600).pack(pady=10)
        
        # Physical offset configuration
        offset_frame = ttk.LabelFrame(parent, text="Physical Offset (meters)")
        offset_frame.pack(fill='x', padx=10, pady=20)
        
        offset_grid = ttk.Frame(offset_frame)
        offset_grid.pack(pady=10)
        
        ttk.Label(offset_grid, text="X (horizontal):").grid(row=0, column=0, padx=5, sticky='w')
        self.offset_x_var = tk.DoubleVar(value=self.config['physical_offset']['x'])
        ttk.Entry(offset_grid, textvariable=self.offset_x_var, width=10).grid(row=0, column=1, padx=5)
        
        ttk.Label(offset_grid, text="Y (depth):").grid(row=0, column=2, padx=5, sticky='w')
        self.offset_y_var = tk.DoubleVar(value=self.config['physical_offset']['y'])
        ttk.Entry(offset_grid, textvariable=self.offset_y_var, width=10).grid(row=0, column=3, padx=5)
        
        ttk.Label(offset_grid, text="Z (vertical):").grid(row=1, column=0, padx=5, sticky='w')
        self.offset_z_var = tk.DoubleVar(value=self.config['physical_offset']['z'])
        ttk.Entry(offset_grid, textvariable=self.offset_z_var, width=10).grid(row=1, column=1, padx=5)
        
        # Render settings
        render_frame = ttk.LabelFrame(parent, text="Render Settings")
        render_frame.pack(fill='x', padx=10, pady=20)
        
        render_grid = ttk.Frame(render_frame)
        render_grid.pack(pady=10)
        
        ttk.Label(render_grid, text="Resolution:").grid(row=0, column=0, padx=5, sticky='w')
        self.res_x_var = tk.IntVar(value=self.config['render_settings']['resolution_x'])
        ttk.Entry(render_grid, textvariable=self.res_x_var, width=8).grid(row=0, column=1, padx=2)
        ttk.Label(render_grid, text="x").grid(row=0, column=2)
        self.res_y_var = tk.IntVar(value=self.config['render_settings']['resolution_y'])
        ttk.Entry(render_grid, textvariable=self.res_y_var, width=8).grid(row=0, column=3, padx=2)
        
        ttk.Label(render_grid, text="FPS:").grid(row=1, column=0, padx=5, sticky='w')
        self.fps_var = tk.IntVar(value=self.config['render_settings']['fps'])
        ttk.Entry(render_grid, textvariable=self.fps_var, width=8).grid(row=1, column=1, padx=2)
        
        # Output directory
        output_frame = ttk.Frame(parent)
        output_frame.pack(fill='x', padx=10, pady=20)
        
        ttk.Label(output_frame, text="Output Directory:").pack(anchor='w')
        
        output_entry_frame = ttk.Frame(output_frame)
        output_entry_frame.pack(fill='x', pady=5)
        
        self.output_var = tk.StringVar(value=self.config['output_dir'])
        ttk.Entry(output_entry_frame, textvariable=self.output_var).pack(side='left', fill='x', expand=True)
        ttk.Button(output_entry_frame, text="Browse", 
                  command=self.browse_output_dir).pack(side='right', padx=(5,0))
    
    def setup_processing_tab(self, parent):
        """Setup processing controls."""
        
        # Processing pipeline status
        ttk.Label(parent, text="Processing Pipeline:").pack(pady=10)
        
        self.steps_frame = ttk.Frame(parent)
        self.steps_frame.pack(fill='x', padx=20, pady=10)
        
        self.processing_steps = [
            ("Load and analyze cameras", False),
            ("Extract tracking data", False),
            ("Synchronize camera timing", False),
            ("Generate Blender script", False),
            ("Setup Blender scene", False),
            ("Render animation", False),
            ("Compile final video", False)
        ]
        
        self.step_vars = []
        for i, (step, completed) in enumerate(self.processing_steps):
            var = tk.BooleanVar(value=completed)
            self.step_vars.append(var)
            
            step_frame = ttk.Frame(self.steps_frame)
            step_frame.pack(fill='x', pady=2)
            
            ttk.Checkbutton(step_frame, variable=var, state='disabled').pack(side='left')
            ttk.Label(step_frame, text=step).pack(side='left', padx=(5,0))
        
        # Progress bar
        ttk.Label(parent, text="Progress:").pack(pady=(20,5))
        self.progress_var = tk.DoubleVar()
        self.progress_bar = ttk.Progressbar(parent, variable=self.progress_var, maximum=100)
        self.progress_bar.pack(fill='x', padx=20, pady=5)
        
        # Processing log
        ttk.Label(parent, text="Processing Log:").pack(pady=(20,5))
        
        log_frame = ttk.Frame(parent)
        log_frame.pack(fill='both', expand=True, padx=10, pady=5)
        
        self.log_text = tk.Text(log_frame, height=15, wrap='word')
        log_scrollbar = ttk.Scrollbar(log_frame, orient='vertical', command=self.log_text.yview)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)
        
        self.log_text.pack(side='left', fill='both', expand=True)
        log_scrollbar.pack(side='right', fill='y')
        
        # Control buttons
        button_frame = ttk.Frame(parent)
        button_frame.pack(pady=20)
        
        self.start_btn = ttk.Button(button_frame, text="Start Processing", 
                                   command=self.start_processing)
        self.start_btn.pack(side='left', padx=5)
        
        self.preview_btn = ttk.Button(button_frame, text="Preview in Blender", 
                                     command=self.preview_in_blender)
        self.preview_btn.pack(side='left', padx=5)
        
        self.stop_btn = ttk.Button(button_frame, text="Stop Processing", 
                                  command=self.stop_processing, state='disabled')
        self.stop_btn.pack(side='left', padx=5)
    
    def setup_results_tab(self, parent):
        """Setup results display."""
        
        ttk.Label(parent, text="Processing Results:").pack(pady=10)
        
        self.results_text = tk.Text(parent, height=8, wrap='word')
        self.results_text.pack(fill='x', padx=10, pady=10)
        
        # Generated files
        ttk.Label(parent, text="Generated Files:").pack(pady=(20,5))
        
        files_frame = ttk.Frame(parent)
        files_frame.pack(fill='both', expand=True, padx=10, pady=5)
        
        files_columns = ('size', 'type', 'modified')
        self.files_tree = ttk.Treeview(files_frame, columns=files_columns, show='tree headings')
        self.files_tree.heading('#0', text='File')
        self.files_tree.heading('size', text='Size')
        self.files_tree.heading('type', text='Type')
        self.files_tree.heading('modified', text='Modified')
        
        files_scrollbar = ttk.Scrollbar(files_frame, orient='vertical', command=self.files_tree.yview)
        self.files_tree.configure(yscrollcommand=files_scrollbar.set)
        
        self.files_tree.pack(side='left', fill='both', expand=True)
        files_scrollbar.pack(side='right', fill='y')
        
        # Action buttons
        action_frame = ttk.Frame(parent)
        action_frame.pack(pady=10)
        
        ttk.Button(action_frame, text="Open Output Folder", 
                  command=self.open_output_folder).pack(side='left', padx=5)
        ttk.Button(action_frame, text="Play Result", 
                  command=self.play_result).pack(side='left', padx=5)
    
    # Video management methods
    def add_video_directories(self):
        """Add video directories."""
        dirs = select_video_dirs(max_dirs=5, parent=self.root)
        for d in dirs:
            if d not in self.video_directories:
                self.video_directories.append(d)
                self.dir_listbox.insert(tk.END, d)
        
        if dirs:
            self.log(f"Added {len(dirs)} video directories")
    
    def scan_videos(self):
        """Scan directories for videos."""
        if not self.video_directories:
            messagebox.showwarning("Warning", "Please add video directories first")
            return
        
        self.log("Scanning for videos...")
        self.available_videos = list_videos_in_dirs(self.video_directories)
        
        # Clear and populate video tree
        for item in self.video_tree.get_children():
            self.video_tree.delete(item)
        
        for video_path in self.available_videos:
            try:
                camera = create_camera(video_path)
                filename = os.path.basename(video_path)
                
                if camera.metadata:
                    m = camera.metadata
                    self.video_tree.insert('', 'end', text=filename, values=(
                        camera.__class__.__name__,
                        f"{m['duration']:.1f}s",
                        f"{m['width']}x{m['height']}",
                        f"{m['fps']:.1f}"
                    ))
                else:
                    self.video_tree.insert('', 'end', text=filename, values=(
                        camera.__class__.__name__, "N/A", "N/A", "N/A"
                    ))
            except Exception as e:
                self.log(f"Error processing {video_path}: {e}")
        
        self.log(f"Found {len(self.available_videos)} videos")
    
    def clear_videos(self):
        """Clear all videos and directories."""
        self.video_directories.clear()
        self.available_videos.clear()
        self.dir_listbox.delete(0, tk.END)
        
        for item in self.video_tree.get_children():
            self.video_tree.delete(item)
        
        self.selected_cameras = {'tracking': None, 'main': None}
        self.tracking_info_var.set("No tracking camera selected")
        self.main_info_var.set("No main camera selected")
        
        self.log("Cleared all videos")
    
    def set_tracking_camera(self):
        """Set selected video as tracking camera."""
        selection = self.video_tree.selection()
        if not selection:
            messagebox.showwarning("Warning", "Please select a video first")
            return
        
        item = selection[0]
        filename = self.video_tree.item(item, 'text')
        video_path = next((v for v in self.available_videos if os.path.basename(v) == filename), None)
        
        if video_path:
            camera = create_camera(video_path)
            self.selected_cameras['tracking'] = camera
            self.tracking_info_var.set(camera.get_info_string())
            self.log(f"Set tracking camera: {filename}")
    
    def set_main_camera(self):
        """Set selected video as main camera."""
        selection = self.video_tree.selection()
        if not selection:
            messagebox.showwarning("Warning", "Please select a video first")
            return
        
        item = selection[0]
        filename = self.video_tree.item(item, 'text')
        video_path = next((v for v in self.available_videos if os.path.basename(v) == filename), None)
        
        if video_path:
            camera = create_camera(video_path)
            self.selected_cameras['main'] = camera
            self.main_info_var.set(camera.get_info_string())
            self.log(f"Set main camera: {filename}")
    
    def analyze_selected_video(self):
        """Analyze the selected video in detail."""
        selection = self.video_tree.selection()
        if not selection:
            messagebox.showwarning("Warning", "Please select a video first")
            return
        
        item = selection[0]
        filename = self.video_tree.item(item, 'text')
        video_path = next((v for v in self.available_videos if os.path.basename(v) == filename), None)
        
        if video_path:
            self.log(f"Analyzing {filename}...")
            camera = create_camera(video_path)
            
            info = f"File: {filename}\n"
            info += f"Type: {camera.__class__.__name__}\n"
            info += f"Path: {video_path}\n"
            
            if camera.metadata:
                m = camera.metadata
                info += f"Resolution: {m['width']}x{m['height']}\n"
                info += f"FPS: {m['fps']:.2f}\n"
                info += f"Duration: {m['duration']:.2f} seconds\n"
                info += f"Frame Count: {m['frame_count']}\n"
            
            # Show in popup
            popup = tk.Toplevel(self.root)
            popup.title(f"Video Analysis: {filename}")
            popup.geometry("400x300")
            
            text_widget = tk.Text(popup, wrap='word')
            text_widget.pack(fill='both', expand=True, padx=10, pady=10)
            text_widget.insert(1.0, info)
            text_widget.config(state='disabled')
    
    def browse_output_dir(self):
        """Browse for output directory."""
        dirname = filedialog.askdirectory(parent=self.root, title="Select output directory")
        if dirname:
            self.output_var.set(dirname)
            self.config['output_dir'] = dirname
    

    def start_processing(self):
        """Start the integrated processing pipeline."""
        if self.is_processing:
            self.log("Processing already in progress!")
            return
        
        # Validate setup
        if not self.selected_cameras['tracking'] or not self.selected_cameras['main']:
            messagebox.showerror("Error", "Please select both tracking and main cameras")
            return
        
        # Update config from UI
        self.update_config_from_ui()
        
        # Start processing in background thread
        self.is_processing = True
        self.start_btn.config(state='disabled')
        self.stop_btn.config(state='normal')
        
        processing_thread = threading.Thread(target=self.process_pipeline)
        processing_thread.daemon = True
        processing_thread.start()
        
        # Start monitoring thread
        self.root.after(100, self.check_processing_queue)
    
    def process_pipeline(self):
        """Main processing pipeline - runs in background thread."""
        try:
            total_steps = len(self.processing_steps)
            
            # Step 1: Load and analyze cameras
            self.queue_update("log", "Step 1: Loading and analyzing cameras...")
            self.queue_update("progress", 1/total_steps * 100)
            self.queue_update("step", 0)
            
            tracking_camera = self.selected_cameras['tracking']
            main_camera = self.selected_cameras['main']
            
            # Step 2: Extract tracking data
            self.queue_update("log", "Step 2: Extracting tracking data...")
            self.queue_update("progress", 2/total_steps * 100)
            self.queue_update("step", 1)
            
            tracking_data = tracking_camera.extract_tracking_data()
            if not tracking_data:
                raise Exception("Failed to extract tracking data")
            
            # Step 3: Synchronize camera timing
            self.queue_update("log", "Step 3: Synchronizing camera timing...")
            self.queue_update("progress", 3/total_steps * 100)
            self.queue_update("step", 2)
            
            synced_data = self.synchronize_cameras(tracking_data, main_camera)
            
            # Step 4: Generate Blender script
            self.queue_update("log", "Step 4: Generating Blender script...")
            self.queue_update("progress", 4/total_steps * 100)
            self.queue_update("step", 3)
            
            script_path = self.generate_blender_script(synced_data)
            
            # Step 5: Setup Blender scene
            self.queue_update("log", "Step 5: Setting up Blender scene...")
            self.queue_update("progress", 5/total_steps * 100)
            self.queue_update("step", 4)
            
            self.setup_blender_scene(script_path)
            
            # Step 6: Render animation
            self.queue_update("log", "Step 6: Rendering animation...")
            self.queue_update("progress", 6/total_steps * 100)
            self.queue_update("step", 5)
            
            self.render_animation(script_path)
            
            # Step 7: Compile final video
            self.queue_update("log", "Step 7: Compiling final video...")
            self.queue_update("progress", 7/total_steps * 100)
            self.queue_update("step", 6)
            
            self.compile_final_video()
            
            self.queue_update("log", "Processing completed successfully!")
            self.queue_update("progress", 100)
            self.queue_update("complete", True)
            
        except Exception as e:
            self.queue_update("log", f"Error during processing: {e}")
            self.queue_update("error", str(e))
        finally:
            self.queue_update("finished", True)
    
    def queue_update(self, update_type, data):
        """Queue UI updates from background thread."""
        self.process_queue.put((update_type, data))
    
    def check_processing_queue(self):
        """Check for updates from processing thread."""
        try:
            while True:
                update_type, data = self.process_queue.get_nowait()
                
                if update_type == "log":
                    self.log(data)
                elif update_type == "progress":
                    self.progress_var.set(data)
                elif update_type == "step":
                    self.step_vars[data].set(True)
                elif update_type == "complete":
                    self.show_results()
                elif update_type == "error":
                    messagebox.showerror("Processing Error", data)
                elif update_type == "finished":
                    self.is_processing = False
                    self.start_btn.config(state='normal')
                    self.stop_btn.config(state='disabled')
                
        except queue.Empty:
            pass
        
        if self.is_processing:
            self.root.after(100, self.check_processing_queue)
    
    def synchronize_cameras(self, tracking_data, main_camera):
        """Synchronize tracking data with main camera timing."""
        if not main_camera.metadata:
            return tracking_data
        
        main_fps = main_camera.metadata['fps']
        main_frame_count = main_camera.metadata['frame_count']
        
        # Resample tracking data to match main camera timing
        synced_frames = []
        for main_frame in range(main_frame_count):
            main_timestamp = main_frame / main_fps
            
            # Find closest tracking frame
            closest_tracking = min(
                tracking_data['frames'],
                key=lambda x: abs(x['timestamp'] - main_timestamp)
            )
            
            # Apply physical offset
            offset = self.config['physical_offset']
            adjusted_position = {
                'x': closest_tracking['position']['x'] + offset['x'],
                'y': closest_tracking['position']['y'] + offset['y'],
                'z': closest_tracking['position']['z'] + offset['z']
            }
            
            synced_frames.append({
                'frame': main_frame,
                'timestamp': main_timestamp,
                'position': adjusted_position,
                'rotation': closest_tracking['rotation'],
                'confidence': closest_tracking.get('confidence', 1.0)
            })
        
        return {
            'frames': synced_frames,
            'source': tracking_data['source'],
            'main_camera': main_camera.video_path
        }
    
    def generate_blender_script(self, synced_data):
        """Generate Blender script for piggyback rig."""
        output_dir = Path(self.config['output_dir'])
        output_dir.mkdir(exist_ok=True)
        
        script_content = f'''
import bpy
import mathutils
import os

# Clear existing scene
bpy.ops.object.select_all(action='SELECT')
bpy.ops.object.delete(use_global=False)

# Synced tracking data
tracking_frames = {synced_data['frames']}
main_camera_path = r"{synced_data['main_camera']}"

print(f"Setting up piggyback rig with {{len(tracking_frames)}} frames")

# Create piggyback rig parent (represents the physical mount)
bpy.ops.object.empty_add(type='PLAIN_AXES', location=(0, 0, 0))
piggyback_rig = bpy.context.object
piggyback_rig.name = "Piggyback_Rig"

# Create tracking camera (top position - invisible)
bpy.ops.object.camera_add(location=(0, 0, 0))
tracking_camera = bpy.context.object
tracking_camera.name = "Tracking_Camera"
tracking_camera.parent = piggyback_rig

# Create main camera (bottom position with offset)
offset = {self.config['physical_offset']}
bpy.ops.object.camera_add(location=(offset['x'], offset['y'], offset['z']))
main_camera = bpy.context.object
main_camera.name = "Main_Camera"
main_camera.parent = piggyback_rig

# Set main camera as render camera
bpy.context.scene.camera = main_camera

# Set scene timing
bpy.context.scene.frame_start = 1
bpy.context.scene.frame_end = len(tracking_frames)
bpy.context.scene.render.fps = {self.config['render_settings']['fps']}

# Apply tracking data to piggyback rig
for i, frame_data in enumerate(tracking_frames):
    frame_num = i + 1
    bpy.context.scene.frame_set(frame_num)
    
    # Move entire piggyback rig
    pos = frame_data['position']
    rot = frame_data['rotation']
    
    # Position
    piggyback_rig.location = (pos['x'], pos['y'], pos['z'])
    piggyback_rig.keyframe_insert(data_path="location")
    
    # Rotation (convert degrees to radians)
    piggyback_rig.rotation_euler = (
        rot['x'] * 3.14159 / 180,
        rot['y'] * 3.14159 / 180,
        rot['z'] * 3.14159 / 180
    )
    piggyback_rig.keyframe_insert(data_path="rotation_euler")

# Add main camera footage as video texture
if os.path.exists(main_camera_path):
    # Create video display plane
    bpy.ops.mesh.primitive_plane_add(location=(0, 3, 0), scale=(3, 1.69, 1))
    video_plane = bpy.context.object
    video_plane.name = "Main_Footage_Display"
    
    # Video material setup
    mat = bpy.data.materials.new(name="MainFootageMaterial")
    mat.use_nodes = True
    video_plane.data.materials.append(mat)
    
    nodes = mat.node_tree.nodes
    tex_node = nodes.new('ShaderNodeTexImage')
    
    # Load main camera video
    img = bpy.data.images.load(main_camera_path)
    img.source = 'MOVIE'
    tex_node.image = img
    
    # Emission shader for video
    emission = nodes.new('ShaderNodeEmission')
    mat.node_tree.links.new(tex_node.outputs[0], emission.inputs[0])
    mat.node_tree.links.new(emission.outputs[0], nodes['Material Output'].inputs[0])

# Scene elements for reference
bpy.ops.mesh.primitive_cube_add(location=(3, 5, 0))
reference_cube = bpy.context.object
reference_cube.name = "Reference_Object"

# Render settings
bpy.context.scene.render.filepath = '{output_dir}/piggyback_render_'
bpy.context.scene.render.image_settings.file_format = 'PNG'
bpy.context.scene.render.resolution_x = {self.config['render_settings']['resolution_x']}
bpy.context.scene.render.resolution_y = {self.config['render_settings']['resolution_y']}

# Set camera field of view (adjust as needed)
main_camera.data.lens = 35

print("Piggyback rig setup complete!")
'''
        
        script_path = output_dir / "piggyback_script.py"
        with open(script_path, 'w') as f:
            f.write(script_content)
        
        return script_path
    
    def setup_blender_scene(self, script_path):
        """Setup Blender scene without rendering."""
        cmd = [
            self.config['blender_path'],
            '--background',
            '--python', str(script_path)
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode != 0:
            raise Exception(f"Blender setup failed: {result.stderr}")
        
        return True
    
    def render_animation(self, script_path):
        """Render animation frames."""
        cmd = [
            self.config['blender_path'],
            '--background',
            '--python', str(script_path),
            '--render-anim'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=3600)
        
        if result.returncode != 0:
            raise Exception(f"Blender render failed: {result.stderr}")
        
        return True
    
    def compile_final_video(self):
        """Compile rendered frames into final video."""
        output_dir = Path(self.config['output_dir'])
        frame_pattern = output_dir / "piggyback_render_*.png"
        output_video = output_dir / "final_piggyback_video.mp4"
        
        # Use ffmpeg to compile frames
        cmd = [
            'ffmpeg', '-y',
            '-framerate', str(self.config['render_settings']['fps']),
            '-pattern_type', 'glob',
            '-i', str(frame_pattern),
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            str(output_video)
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)
            if result.returncode != 0:
                raise Exception(f"FFmpeg compilation failed: {result.stderr}")
        except FileNotFoundError:
            self.log("FFmpeg not found - frames available but not compiled to video")
        
        return True
    
    def preview_in_blender(self):
        """Open Blender with preview setup."""
        if not self.selected_cameras['tracking'] or not self.selected_cameras['main']:
            messagebox.showwarning("Warning", "Please select both cameras first")
            return
        
        self.log("Generating preview Blender script...")
        
        # Generate quick preview data
        tracking_data = {'frames': [
            {'frame': i, 'timestamp': i/30, 'position': {'x': 0, 'y': 0, 'z': 0}, 
             'rotation': {'x': 0, 'y': 0, 'z': 0}} for i in range(100)
        ]}
        
        synced_data = self.synchronize_cameras(tracking_data, self.selected_cameras['main'])
        script_path = self.generate_blender_script(synced_data)
        
        # Open Blender with the script
        cmd = [self.config['blender_path'], '--python', str(script_path)]
        subprocess.Popen(cmd)
        
        self.log("Blender opened with preview setup")
    
    def stop_processing(self):
        """Stop processing (placeholder)."""
        self.log("Stop processing requested...")
        # In a real implementation, you'd set a flag to stop the processing thread
    
    def show_results(self):
        """Display processing results."""
        output_dir = Path(self.config['output_dir'])
        
        results = "Processing completed successfully!\n\n"
        results += f"Output directory: {output_dir}\n\n"
        results += "Generated files:\n"
        
        # List generated files
        for item in self.files_tree.get_children():
            self.files_tree.delete(item)
        
        if output_dir.exists():
            for file_path in output_dir.iterdir():
                if file_path.is_file():
                    stat = file_path.stat()
                    size = f"{stat.st_size / 1024 / 1024:.2f} MB"
                    file_type = file_path.suffix[1:].upper() if file_path.suffix else "Unknown"
                    modified = datetime.fromtimestamp(stat.st_mtime).strftime("%Y-%m-%d %H:%M")
                    
                    self.files_tree.insert('', 'end', text=file_path.name, 
                                         values=(size, file_type, modified))
                    results += f"- {file_path.name}\n"
        
        self.results_text.delete(1.0, tk.END)
        self.results_text.insert(1.0, results)
    
    def open_output_folder(self):
        """Open output folder in file explorer."""
        output_path = Path(self.config['output_dir'])
        if output_path.exists():
            if os.name == 'nt':  # Windows
                os.startfile(output_path)
            elif os.name == 'posix':  # macOS and Linux
                subprocess.run(['open' if sys.platform == 'darwin' else 'xdg-open', str(output_path)])
    
    def play_result(self):
        """Play the result video."""
        output_dir = Path(self.config['output_dir'])
        video_file = output_dir / "final_piggyback_video.mp4"
        
        if video_file.exists():
            if os.name == 'nt':  # Windows
                os.startfile(video_file)
            elif os.name == 'posix':  # macOS and Linux
                subprocess.run(['open' if sys.platform == 'darwin' else 'xdg-open', str(video_file)])
        else:
            messagebox.showinfo("Info", "Final video not found. Check the output folder for rendered frames.")
    
    def update_config_from_ui(self):
        """Update configuration from UI values."""
        self.config['output_dir'] = self.output_var.get()
        
        self.config['physical_offset'] = {
            'x': self.offset_x_var.get(),
            'y': self.offset_y_var.get(),
            'z': self.offset_z_var.get()
        }
        
        self.config['render_settings'] = {
            'resolution_x': self.res_x_var.get(),
            'resolution_y': self.res_y_var.get(),
            'fps': self.fps_var.get(),
            'format': 'PNG'
        }
    
    def log(self, message):
        """Add message to processing log."""
        timestamp = datetime.now().strftime("%H:%M:%S")
        log_message = f"[{timestamp}] {message}\n"
        
        self.log_text.insert(tk.END, log_message)
        self.log_text.see(tk.END)
        self.root.update_idletasks()
        
def get_video_file_interactive():
    """
    Handles video file selection across notebook, iOS/Juno, and CLI/desktop.
    Returns: path to video file (str), or None if not selected yet.
    """
    import sys
    import os

    # 1. Try Jupyter Notebook upload widget
    try:
        from IPython.display import display
        import ipywidgets as widgets
        upload_widget = widgets.FileUpload(accept='.mp4,.mov,.avi,.mkv', multiple=False)
        display(upload_widget)
        print("⬆️ Upload your video file above, then rerun this cell when finished.")
        return None
    except Exception:
        pass

    # 2. Fallback: On iOS/Juno: auto-select most recent video in working dir
    cwd = os.getcwd()
    files = [f for f in os.listdir(cwd) if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
    if files:
        files.sort(key=lambda x: os.path.getmtime(os.path.join(cwd, x)), reverse=True)
        latest = files[0]
        print(f"📹 Auto-selected latest video: {latest}")
        return os.path.join(cwd, latest)

    # 3. CLI fallback: manual entry
    print("No upload widget or video found. Enter full path to your video file (.mp4/.mov/.avi/.mkv):")
    path = input("Video file path: ").strip()
    if os.path.isfile(path):
        return path

    print("❌ No video file found or selected.")
    return None

def main():
    mode = get_app_mode()
    print(f"Launching in {mode} mode...")

    if mode == 'tkinter':
        tk, ttk, filedialog, messagebox = try_import_tkinter()
        root = tk.Tk()
        app = IntegratedPiggybackApp(root)
        root.mainloop()
    elif mode == 'jupyter':
        # Notebook fallback: prompt for upload, list recent, etc
        print("Notebook mode detected. Using minimal video selection.")
        video_path = get_video_file_interactive()
        if not video_path:
            print("Upload a video in the widget above, then rerun this cell.")
        else:
            print(f"Using: {video_path}")
            # Optionally: instantiate AbstractCamera subclass and run analysis
    else:
        # CLI/minimal fallback
        print("No GUI available. Using command-line selection.")
        cwd = os.getcwd()
        files = [f for f in os.listdir(cwd) if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
        if not files:
            print("No videos found in current directory.")
            sys.exit(1)
        for i, fname in enumerate(files):
            print(f"[{i}] {fname}")
        idx = int(input("Select a video by number: "))
        video_path = os.path.join(cwd, files[idx])
        print(f"Selected: {video_path}")
        # Optionally: process video_path with your camera class

if __name__ == "__main__":
    main()

Launching in jupyter mode...
Notebook mode detected. Using minimal video selection.


FileUpload(value=(), accept='.mp4,.mov,.avi,.mkv', description='Upload')

⬆️ Upload your video file above, then rerun this cell when finished.
Upload a video in the widget above, then rerun this cell.


### The Following Cell Loads A Video

In [3]:
import ipywidgets as widgets
from IPython.display import display

upload_widget = widgets.FileUpload(accept='.mp4,.mov,.avi,.mkv', multiple=False)
display(upload_widget)

def save_uploaded_video(upload_widget):
    # Handle tuple (ipywidgets >= 8) or dict (older)
    if isinstance(upload_widget.value, dict):
        files = upload_widget.value.items()
    elif isinstance(upload_widget.value, tuple):
        files = [(f['name'], f) for f in upload_widget.value]
    else:
        raise ValueError("Unknown upload_widget.value type")

    for name, fileinfo in files:
        with open(name, "wb") as f:
            f.write(fileinfo['content'])
        print(f"Saved: {name}")
        return name  # Only one file, so return the first

FileUpload(value=(), accept='.mp4,.mov,.avi,.mkv', description='Upload')

In [2]:
# Usage:
# After uploading file, run:
video_path = save_uploaded_video(upload_widget)

Saved: 20250606_2322_Video Metadata Workflow_simple_compose_01jx46jcy3fhy8xkgexn2zvnnz.mp4


# Generating Keywords and Description

In [3]:
import cv2
import os
from datetime import datetime
from collections import Counter

def extract_keyframes(video_path, every_n_seconds=2, max_frames=12):
    """
    Extracts key frames from video at N-second intervals.
    Returns list of frame file paths.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = int(fps * every_n_seconds)
    frames = []
    saved = 0
    for i in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret: break
        fname = f"frame_{i}.jpg"
        cv2.imwrite(fname, frame)
        frames.append(fname)
        saved += 1
        if saved >= max_frames:
            break
    cap.release()
    return frames

def describe_image(image_path, model="gpt4v"):
    """
    Describe an image using a vision model.
    For local: use BLIP/CLIP; for OpenAI, call their API.
    """
    # --- Example: BLIP (Huggingface, local, no API key needed) ---
    try:
        from transformers import BlipProcessor, BlipForConditionalGeneration
        from PIL import Image
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        img = Image.open(image_path).convert('RGB')
        inputs = processor(img, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        return f"Error describing image: {e}"

def analyze_video(video_path):
    keyframes = extract_keyframes(video_path)
    captions = []
    print(f"Extracted {len(keyframes)} keyframes.")
    for frame in keyframes:
        desc = describe_image(frame)
        print(f"{frame}: {desc}")
        captions.append(desc)
    # Generate keywords
    words = []
    for c in captions:
        words += [w.lower() for w in c.replace('.', '').split()]
    keywords = [w for w, _ in Counter(words).most_common() if len(w) > 3]
    summary = ". ".join(captions)
    return {
        "description": summary,
        "keywords": keywords[:10]
    }

# Usage Example
if __name__ == "__main__":
    video_path = "./20250606_2322_Video Metadata Workflow_simple_compose_01jx46jcy3fhy8xkgexn2zvnnz.mp4"
    result = analyze_video(video_path)
    print("=== Description ===")
    print(result["description"])
    print("=== Keywords ===")
    print(result["keywords"])

AttributeError: module 'cv2' has no attribute 'VideoCapture'

In [3]:
import cv2
print(cv2.__version__)
print(hasattr(cv2, 'VideoCapture'))  # should return True

4.10.0
False


In [4]:
from moviepy.editor import VideoFileClip

clip = VideoFileClip("your_video.mp4")
frame = clip.get_frame(5.0)  # Get frame at 5s (returns a numpy array)

AttributeError: 'NoneType' object has no attribute 'startswith'

In [2]:
if hasattr(cv2, 'VideoCapture'):
    # extract from video
else:
    # fallback to reading local .jpg files

IndentationError: expected an indented block after 'if' statement on line 1 (<ipython-input-2-7b4adbe6f53a>, line 3)

In [1]:
import cv2
import os

frames = [f for f in os.listdir('.') if f.endswith('.jpg')]
for f in frames:
    img = cv2.imread(f)
    # analyze or describe `img` here

In [3]:
import imageio
imageio.plugins.ffmpeg.download()

RuntimeError: imageio.ffmpeg.download() has been deprecated. Use 'pip install imageio-ffmpeg' instead.'

In [2]:
import imageio_ffmpeg
import os

# Manually point to a bundled ffmpeg (if you can get one copied into the app sandbox)
os.environ["IMAGEIO_FFMPEG_EXE"] = "/path/to/your/ffmpeg"

In [3]:
from moviepy.editor import VideoFileClip

def extract_frame_fallback(video_path, time_sec=1.0):
    try:
        clip = VideoFileClip(video_path)
        frame = clip.get_frame(time_sec)
        return frame  # returns a numpy array (image)
    except Exception as e:
        print("MoviePy failed to extract frame:", e)
        return None

NameError: name 'frame' is not defined

# Moving on...

Here’s what to put in a notebook cell to run the complete piggyback camera app:

```python
# Run the Integrated Piggyback Camera App
if __name__ == "__main__":
    # Check if we're in a notebook environment
    try:
        get_ipython()
        print("Running in Jupyter notebook - launching desktop app...")
        
        # For notebook environments, run in a separate thread to avoid blocking
        import threading
        
        def launch_app():
            root = tk.Tk()
            app = IntegratedPiggybackApp(root)
            root.mainloop()
        
        app_thread = threading.Thread(target=launch_app, daemon=True)
        app_thread.start()
        
        print("✅ Piggyback Camera App launched!")
        print("📋 Quick Start Guide:")
        print("1. Go to 'Video Selection' tab")
        print("2. Click 'Add Video Directories' to select folders with your videos")
        print("3. Click 'Scan for Videos' to find all video files")
        print("4. Select your Insta360 tracking video and click 'Set as Tracking Camera'")
        print("5. Select your main camera video and click 'Set as Main Camera'")
        print("6. Go to 'Camera Setup' tab to configure physical offset")
        print("7. Go to 'Processing' tab and click 'Start Processing'")
        print("8. Check 'Results' tab when complete!")
        
    except NameError:
        # Not in notebook, run normally
        main()
```

**Or for a simpler direct launch:**

```python
# Direct launch (blocks notebook until app is closed)
main()
```

**For testing with sample data:**

```python
# Quick test with sample videos (if you have them in current directory)
import os

# Check what videos are available in current directory
videos_here = [f for f in os.listdir('.') if f.lower().endswith(('.mp4', '.mov', '.avi', '.invis'))]
if videos_here:
    print("Found videos in current directory:")
    for v in videos_here:
        print(f"  📹 {v}")
    print("\nLaunching app...")
    main()
else:
    print("No videos found in current directory.")
    print("Place some video files here or use the app's directory browser.")
    main()
```

**For automated setup (if you know your file paths):**

```python
# Pre-configure and launch
root = tk.Tk()
app = IntegratedPiggybackApp(root)

# Pre-set some directories if you know them
known_video_dirs = [
    "/path/to/your/video/folder1",
    "/path/to/your/video/folder2"
]

for directory in known_video_dirs:
    if os.path.exists(directory):
        app.video_directories.append(directory)
        app.dir_listbox.insert(tk.END, directory)

print("🚀 App launched with pre-configured directories!")
root.mainloop()
```

The first option is best for notebook use since it runs the GUI in a separate thread and gives you helpful instructions!​​​​​​​​​​​​​​​​